In [11]:
pip install requests bs4

Note: you may need to restart the kernel to use updated packages.


In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import json

In [27]:
listlocal=[]
listprecio=[]
listarea=[]
listdes1=[]
listlat=[]
listlong=[]
listdes2=[]
for y in range(1,7):
    URL = 'https://www.vivanuncios.com.mx/s-renta-locales-comerciales/queretaro-qro/page-{}/v1c1276l11792p{}?q=local+para+restaurante'
    headers ={"User-Agent" : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:82.0) Gecko/20100101 Firefox/82.0'}
    page = requests.get(URL.format(y,y), headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    locales = soup.findAll("div", {"class" : "tileV2 REAdTileV2 regular mapView"})
    ubicaciones = soup.find("script", type = "application/ld+json")
    geo=json.loads(ubicaciones.string)
    
    for x in range(0,len(geo)):
        try:
            lat=geo[x]["geo"]["latitude"]
            long=geo[x]["geo"]["longitude"]
            des2=geo[x]["name"]
            listlat.append(lat)
            listlong.append(long)
            listdes2.append(des2)
        except KeyError:
            pass;
    
    for local in locales: 
        try:
            local_ubic = local.findAll("div",{"class" : "tile-location one-liner"})
            lu=local_ubic[0].text.strip()
            listlocal.append(lu)
        except IndexError:
            lu="-"
            listlocal.append(lu)
        try: 
            local_precio = local.findAll("span",{"class" : "ad-price"})
            lp = local_precio[0].text.strip()
            lp2=lp.replace("USD","")
            lp3=lp2.replace("$","")
            lp4=lp3.strip()
            lp5=int(lp4.replace(",",""))
            listprecio.append(lp5)
            #listprecio.append(lp)
        except IndexError:
            lp5=""
            #lp="-"
            listprecio.append(lp5)
            #listprecio.append(lp)
        try: 
            local_area = local.findAll("div",{"class" : "chiplets-inline-block surface-area"})
            la=local_area[0].text.strip()
            la2=int(la.strip(" m\u00b2"))
            listarea.append(la2)
            #listarea.append(la)
        except IndexError:
            #la="-"
            #listarea.append(la)
            la2=""
            listarea.append(la2)
        if len(listdes1)>=len(listdes2):
            lat="-"
            long="-"
            des2="-"
            listlat.append(lat)
            listlong.append(long)
            listdes2.append(des2)
        else:
            pass;
        descrip1 = local.findAll("a",{"target" : "_blank"})
        des1=descrip1[0].text.strip()
        listdes1.append(des1)
           
df=pd.DataFrame({"Lugar":listlocal,"Renta":listprecio,"Área":listarea,"Latitud":listlat,"Longitud":listlong,"Descripción":listdes1})
df

,Lugar,Renta,Área,Latitud,Longitud,Descripción
0,El Campanario,60000,160,20.6092,-100.358,LOCAL COMERCIAL EN PRE-RENTA EN EL CAMPANARIO ...
1,El Campanario,38000,110,20.6157,-100.358,RENTA LOCAL COMERCIAL EN LA RESERVA EL CAMPANA...
2,San Pablo,18000,90,20.624,-100.421,Local en Renta en San Pablo
3,Centro Sur,54600,210,20.56,-100.36,RENTA LOCALES Comerciales en QUERETARO Zona Co...
4,Alamos 2da Secc,66000,,20.6129,-100.387,RENTA DE LOCAL CON EXCELENTE UBICACIÓN EN ÁLAMOS
...,...,...,...,...,...,...
110,Colonia,8500,,20.5475,-100.417,LOCAL COMERCIAL EN VENTA O RENTA EN EL PUEBLI...
111,Puerta Real Residencial Desarrollo Urbana 08,8500,,20.563,-100.43,RENTA LOCAL CORREGIDORA
112,El Refugio,50000,,20.6489,-100.346,LOCAL COMERCIAL EN DOS NIVELES
113,El Marqués,35000,,20.5907,-100.358,Planta Baja. DOS FRENTES sobre carretera Quer...


In [28]:
df2 = df[df.Latitud != '-']
df3 = df2[df.Área != '']
df4 = df3[df.Renta != '']

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()


In [29]:
df4

,Lugar,Renta,Área,Latitud,Longitud,Descripción
0,El Campanario,60000,160,20.6092,-100.358,LOCAL COMERCIAL EN PRE-RENTA EN EL CAMPANARIO ...
1,El Campanario,38000,110,20.6157,-100.358,RENTA LOCAL COMERCIAL EN LA RESERVA EL CAMPANA...
2,San Pablo,18000,90,20.624,-100.421,Local en Renta en San Pablo
3,Centro Sur,54600,210,20.56,-100.36,RENTA LOCALES Comerciales en QUERETARO Zona Co...
7,Centro Sur,17000,70,20.5619,-100.363,RENTA Local nuevo en en QUERETARO en Plaza Co...
8,Milenio 3ra Secc,15500,72,20.5942,-100.346,LOCAL EN RENTA CON TERRAZA IDEAL PARA RESTAURA...
9,Cimatario,9926,47,20.584,-100.388,RENTO LOCAL NUEVO EN EL CENTRO
11,Hercules,10000,120,20.5954,-100.35,Renta de Local Comercial Centrico en Hércules ...
15,Mercurio,19000,70,20.5844,-100.38,Local Mercurio a un costado del centro cultura...
17,Carretas,3500,300,20.5888,-100.366,LOCAL EN RENTA


In [30]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [31]:
import folium # plotting library
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [33]:
map_loc_qro = folium.Map(location=[20.580226,-100.398011],zoom_start=13)

for lat,lng,lug,renta,area,desc in zip(df4['Latitud'],df4['Longitud'],df4['Lugar'],df4["Renta"],df4['Área'],df4["Descripción"]):
    label = '{}, {}, {}'.format(lug,renta ,desc)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_loc_qro)
map_loc_qro